In [1]:
%run functions.ipynb

Ready: 2023-05-20 11:16:16


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

spark

In [5]:
rs = "('" + "','".join(list(df_all.rubricName.unique())) + "')"

df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', IntegerType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
df_list = [i for i in files_list_chain]

for file in tqdm(df_list):
    df = spark.read.option("header","true").csv(file, schema=df_schema)
    df.createOrReplaceTempView('df')
    city = file.split('\\')[2]
    df_new = spark.sql(f'''
    select 
        rubricName as rubric
        , "{city}" city
        , user
        , SUBSTR(time, 6, 5) date_r
        from df
    ''').where(f'rubric in {rs}')

    df_new.write.partitionBy('rubric').mode('append').parquet(f'files/ten_city/df_dates_users_city2')

In [3]:
df_all = pd.concat([pd.read_parquet(f'files/ten_city/df_dates_{col_r}').rename({col_r: 'date_r'}, axis=1)[['rubricName', 'date_r', 'cnt', 'city', 'year_r']] for col_r in ['hour_r', 'date_r', 'week_r', 'week_day_r']])
df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubricName']
df_all = df_all[~df_all.rubricName.apply(lambda x: (x is None or x.isnumeric()))]
rubrics = df_all.rubricName.unique()

for rubric in tqdm(rubrics[265:]):
    try:
        df = spark.read.parquet(fr"C:\Users\Mamed\_Notebooks\2GIS_2023\files\ten_city\df_dates_users_city2\rubric={rubric.replace(' ', '%20').replace('/','%2F')}")
        df.createOrReplaceTempView('df')
        df_new = spark.sql(f'''
        select 
            "{rubric}" rubric
            ,city
            , date_r
            ,count(distinct user) cnt_users
            ,count(*) cnt_requests
        from df
            group by 
                    "{rubric}"
                    ,city
                    ,date_r

        ''')
        df_new.write.mode('append').parquet(f'files/ten_city/df_dates_users_city_agg')
    except:
        pass

In [18]:
df1 = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg')
df1['city_rubric_param'] = df1['city'] + '. ' + df1['rubric'] + '. ' + 'Пользователи'
df1 = df1[['date_r', 'cnt_users', 'city_rubric_param']].rename({'cnt_users':'cnt'}, axis=1)

df2 = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg')
df2['city_rubric_param'] = df2['city'] + '. ' + df2['rubric'] + '. ' + 'Запросы'
df2 = df2[['date_r', 'cnt_requests', 'city_rubric_param']].rename({'cnt_requests':'cnt'}, axis=1)
df_dates_users = pd.concat([df1, df2])
df_dates_users['cnt_all'] = df_dates_users.groupby('city_rubric_param')['cnt'].transform(np.sum)
df_dates_users['city_rubric_param_cnt'] = df_dates_users['cnt_all'].astype(str) + '_' + df_dates_users['city_rubric_param']

dfx = pd.pivot_table(df_dates_users, values='cnt', aggfunc='sum', index=['date_r'], columns=['city_rubric_param_cnt'])
t = dfx.corr().fillna(0).astype(np.float16)

In [19]:
t.shape

(24120, 24120)

In [20]:
t.iloc[:10,:10]

city_rubric_param_cnt,100035_Астана. Электроинструмент. Запросы,100066_Новосибирск. Офисная мебель. Запросы,1000_Калининград. Информационная безопасность. Запросы,1000_Москва. Аквариумы. Пользователи,1000_ОАЭ. Обучение по промышленной безопасности. Запросы,10010_Владивосток. Институты. Пользователи,100137_Астана. Кулинарии. Пользователи,10015_Владивосток. Центры виртуальной реальности. Запросы,10016_Астана. Гончарные мастерские. Запросы,10017_СПБ. Прокат оборудования / инструментов. Запросы
city_rubric_param_cnt,,,,,,,,,,
100035_Астана. Электроинструмент. Запросы,1.000000,0.905762,0.295166,0.025101,0.347656,0.777344,0.869141,0.351074,0.412354,0.711914
100066_Новосибирск. Офисная мебель. Запросы,0.905762,1.000000,0.383545,-0.027878,0.245239,0.781738,0.869141,0.362549,0.210571,0.721191
1000_Калининград. Информационная безопасность. Запросы,0.295166,0.383545,1.000000,-0.056549,0.135620,0.352295,0.096313,-0.222412,0.160156,0.183960
1000_Москва. Аквариумы. Пользователи,0.025101,-0.027878,-0.056549,1.000000,0.062408,-0.114929,-0.059784,0.241089,-0.281006,0.042572
1000_ОАЭ. Обучение по промышленной безопасности. Запросы,0.347656,0.245239,0.135620,0.062408,1.000000,0.396240,0.284180,-0.048767,0.061523,0.053650
10010_Владивосток. Институты. Пользователи,0.777344,0.781738,0.352295,-0.114929,0.396240,1.000000,0.754883,0.107605,0.422119,0.409424
100137_Астана. Кулинарии. Пользователи,0.869141,0.869141,0.096313,-0.059784,0.284180,0.754883,1.000000,0.661621,0.298096,0.710938
10015_Владивосток. Центры виртуальной реальности. Запросы,0.351074,0.362549,-0.222412,0.241089,-0.048767,0.107605,0.661621,1.000000,-0.003057,0.343018
10016_Астана. Гончарные мастерские. Запросы,0.412354,0.210571,0.160156,-0.281006,0.061523,0.422119,0.298096,-0.003057,1.000000,0.268066


In [21]:
t = t.rename_axis([None]).rename_axis([None], axis=1)
t1 = t.stack()
t1 = t1.reset_index()
t1.columns = ['r1', 'r2', 'corr']
t1 = t1[t1.r1 != t1.r2]

In [22]:
with open('files/ten_city/t1_new.pkl', 'wb') as fp:
    pickle.dump(t1, fp)

In [23]:
t1.shape

(581750280, 3)

### 1.	Однородность:

In [13]:
df_all = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg')
df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubric']
df_all = df_all[~df_all.rubric.apply(lambda x: (x is None or x.isnumeric()))]
df_all['cnt_all'] = df_all.groupby('city_rubric')['cnt_requests'].transform(np.sum)
df_all = df_all.drop(['date_r', 'cnt_requests', 'cnt_users'], axis=1).drop_duplicates()
df_all['r1_rn_city'] = (df_all.sort_values(['city', 'cnt_all'], ascending=[True,False]).groupby(['city']).cumcount() + 1)
df_all = df_all.sort_values(['cnt_all'], ascending=False).reset_index().reset_index().set_index('level_0').drop('index', axis=1)
df_all['r1_rn_all'] = df_all.index
df_all['cnt_city_rubric'] = df_all['cnt_all'].astype(str) + '_' + df_all['city_rubric'] + '. ' + 'Запросы'
df_1000 = df_all[df_all['r1_rn_city']<=100]
df_list = []
rubric_list = set()
for c in df_1000.city.unique():
    df_i = df_1000[(df_1000['city'] == c) & (~df_1000['rubric'].isin(rubric_list))].iloc[:9]
    df_list.append(df_i)
    _ = [rubric_list.add(i) for i in df_i.rubric.values]
df_100 = pd.concat(df_list)
df_100['r1_rn_city'] = (df_100.sort_values(['city', 'cnt_all'], ascending=[True,False]).groupby(['city']).cumcount() + 1)
df_100_requests = df_100

In [14]:
df_all = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg')
df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubric']
df_all = df_all[~df_all.rubric.apply(lambda x: (x is None or x.isnumeric()))]
df_all['cnt_all'] = df_all.groupby('city_rubric')['cnt_users'].transform(np.sum)
df_all = df_all.drop(['date_r', 'cnt_requests', 'cnt_users'], axis=1).drop_duplicates()
df_all['r1_rn_city'] = (df_all.sort_values(['city', 'cnt_all'], ascending=[True,False]).groupby(['city']).cumcount() + 1)
df_all = df_all.sort_values(['cnt_all'], ascending=False).reset_index().reset_index().set_index('level_0').drop('index', axis=1)
df_all['r1_rn_all'] = df_all.index
df_all['cnt_city_rubric'] = df_all['cnt_all'].astype(str) + '_' + df_all['city_rubric']  + '. ' + 'Пользователи'
df_1000 = df_all[df_all['r1_rn_city']<=100]
df_list = []
rubric_list = set()
for c in df_1000.city.unique():
    df_i = df_1000[(df_1000['city'] == c) & (~df_1000['rubric'].isin(rubric_list))].iloc[:9]
    df_list.append(df_i)
    _ = [rubric_list.add(i) for i in df_i.rubric.values]
df_100 = pd.concat(df_list)
df_100['r1_rn_city'] = (df_100.sort_values(['city', 'cnt_all'], ascending=[True,False]).groupby(['city']).cumcount() + 1)
df_100_users = df_100

In [18]:
df_100 = pd.concat([df_100_requests, df_100_users])

In [4]:
with open('files/ten_city/t1_new.pkl', 'rb') as fp:
    t1 = pickle.load(fp)

In [37]:
t1_100 = df_100.merge(t1, left_on='cnt_city_rubric', right_on='r1')

t1_100['r1_cnt'] = t1_100.r1.apply(lambda x: x.split('_')[0])
t1_100['r2_cnt'] = t1_100.r2.apply(lambda x: x.split('_')[0])

t1_100.r1 = t1_100.r1.apply(lambda x: x.split('_')[1])
t1_100.r2 = t1_100.r2.apply(lambda x: x.split('_')[1])
t1_100['r1_city'] = t1_100.r1.apply(lambda x: x.split('. ')[0])
t1_100['r2_city'] = t1_100.r2.apply(lambda x: x.split('. ')[0])
t1_100['r1_rubric'] = t1_100.r1.apply(lambda x: x.split('. ')[1])
t1_100['r2_rubric'] = t1_100.r2.apply(lambda x: x.split('. ')[1])
t1_100['r1_param'] = t1_100.r1.apply(lambda x: x.split('. ')[2])
t1_100['r2_param'] = t1_100.r2.apply(lambda x: x.split('. ')[2])

In [45]:
t1_100.r1_cnt = t1_100.r1_cnt.astype(int)
t1_100.r2_cnt = t1_100.r2_cnt.astype(int)

In [46]:
t1_100 = t1_100[t1_100.r2_cnt > 1000]

In [ ]:
t1_100 = t1_100[['r1_city', 'r1_rubric', 'r1_param', 'r2_city', 'r2_rubric', 'r2_param', 'corr', 'r1_cnt', 'r2_cnt']]

### f7

#### органическая – для построения тренда используются смысловые ряды переменных: запросы базируются на запросах, пользователи на пользователях, динамика на динамике, схожесть на схожести

In [56]:
df_100_i = df_100[df_100['r1_rn_city'] == 7]
cr = list(zip(df_100_i['city'].values, df_100_i['rubric'].values))

df_list = []
for city, rubric in tqdm(cr):
    t1_c1 = t1_100[(t1_100.r1_city == city) & (t1_100.r1_rubric == rubric) & (t1_100.r1_param == t1_100.r2_param)].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i = t1_c1.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    df_list.append(f1_i)
f7 = pd.concat(df_list).sort_values(['r1_city','r1_rubric','r1_param'])
f7['group'] = 'f7'

  0%|          | 0/18 [00:00<?, ?it/s]

### f8

#### синтетическая – в формулу замешиваются показатели наибольшей предсказательной силы, без учёта их физической природы

In [57]:
df_100_i = df_100[df_100['r1_rn_city'] == 8]
cr = list(zip(df_100_i['city'].values, df_100_i['rubric'].values))

df_list = []
for city, rubric in tqdm(cr):
    t1_c1 = t1_100[(t1_100.r1_city == city) & (t1_100.r1_rubric == rubric)].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i = t1_c1.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    df_list.append(f1_i)
f8 = pd.concat(df_list).sort_values(['r1_city','r1_rubric','r1_param'])
f8['group'] = 'f8'

  0%|          | 0/18 [00:00<?, ?it/s]

### f9

### адаптационная – задействуется оптимизационный алгоритм вариативности, чередующий переменные, не допуская преобладания единственного типа.

In [58]:
df_100_i = df_100[df_100['r1_rn_city'] == 9]
cr = list(zip(df_100_i['city'].values, df_100_i['rubric'].values))

df_list = []
for city, rubric in tqdm(cr):
    t1_c1 = t1_100[(t1_100.r1_city == city) & (t1_100.r1_rubric == rubric) & (t1_100.r2_param == 'Пользователи')].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    t1_c2 = t1_100[(t1_100.r1_city == city) & (t1_100.r1_rubric == rubric) & (t1_100.r2_param == 'Запросы')].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i = pd.concat([t1_c1.iloc[:10], t1_c2.iloc[:10]])
    f1_i['avg_corr'] = f1_i['corr'].mean()
    df_list.append(f1_i)
f9 = pd.concat(df_list)
f9['group'] = 'f9'

  0%|          | 0/18 [00:00<?, ?it/s]

In [61]:
f_all = pd.concat([f7,f8,f9])

In [62]:
f_all['r1_rubric_city'] = f_all['r1_city'] + '. ' + f_all['r1_rubric'] + '. ' + f_all['r1_param'] 
f_all['r2_rubric_city'] = f_all['r2_city'] + '. ' + f_all['r2_rubric'] + '. ' + f_all['r2_param']

In [65]:
f_cr = f_all[['r1_city','r1_rubric','r1_param']].drop_duplicates()
cr = list(zip(f_cr['r1_city'].values, f_cr['r1_rubric'].values, f_cr['r1_param'].values))

In [77]:
len(cr)

68

In [66]:
text = ''

In [67]:
def replace_slash(s):
    return s[::-1].replace('/', 'и', 1)[::-1]

In [74]:
text = ''
for city, rubric, param in cr:
    f_i = f_all[(f_all.r1_city == city) & (f_all.r1_rubric == rubric) & (f_all.r1_param == param)]
    head = f_i['group'].iloc[0] +': '+ f_i['r1_rubric_city'].iloc[0] + ' = f{'
    body = '\n\t,'.join([f'{i:.04f} x {replace_slash(j)}' for i, j in list(zip(f_i['corr'].values, f_i['r2_rubric_city'].values))]) + '}\n'
    text += head+body

In [75]:
print(text)

f7: Астана. Гостиницы. Запросы = f{0.9946 x Астана. Кафе. Запросы
	,0.9912 x Астана. Кофейни. Запросы}
f7: Астана. Гостиницы. Пользователи = f{0.9971 x Астана. Кафе. Пользователи
	,0.9971 x Астана. Кофейни. Пользователи
	,0.9966 x Бишкек. Гостиницы. Пользователи
	,0.9961 x Астана. Супермаркеты. Пользователи
	,0.9956 x Астана. Торговые центры и Универсальные магазины. Пользователи
	,0.9946 x Астана. Рестораны. Пользователи
	,0.9946 x Астана. Продуктовые магазины. Пользователи
	,0.9941 x Астана. Быстрое питание. Пользователи
	,0.9941 x Астана. Женская одежда. Пользователи
	,0.9937 x Бишкек. Быстрое питание. Пользователи
	,0.9937 x Астана. Столовые. Пользователи
	,0.9932 x Бишкек. Кафе. Пользователи
	,0.9927 x Астана. Рынки. Пользователи
	,0.9927 x Владивосток. Продуктовые магазины. Пользователи
	,0.9922 x Бишкек. Торгово-развлекательные центры и Моллы. Пользователи
	,0.9922 x Астана. Заправочные станции. Пользователи
	,0.9922 x Бишкек. Продуктовые магазины. Пользователи
	,0.9917 x Астана